In [1]:
from numpy.random import normal,multivariate_normal
import numpy as np
from numpy.linalg import inv
from scipy import *
from scipy.linalg import norm, pinv, det
from scipy.misc import derivative
from sympy import symbols, diff
import matplotlib.pyplot as plt
import numdifftools as nd


In [2]:
def kmeans(x,k):
    n = x.shape[0]
    n_iterations = 1000
    epsilon = 1e-8
    err = np.inf

    #initialize random centroids
    c = np.zeros((k,x.shape[1]))
    for i in range(k):
        idx = np.random.randint(n)
        c[i] = x[idx,:]

    for iteration in range(n_iterations):
        d = np.zeros((n, k))  # distances to centroids square
        for i in range(k):
            d[:, i] = np.sqrt(np.sum(np.square(np.subtract(x, np.tile(c[i, :], (n, 1)))), 1).flatten())

        #assign each point to nearest centroid
        l = np.argmin(d,1)


        #recompute centroids as center of mass of each cluster
        for j in range(k):
            if len(l[l==j]):
                c[j,:] = x[l==j].sum(0) / len(l[l==j])

        err_new = np.sum(np.min(d,1))/n


        if np.abs(err_new - err)<epsilon:
            break
        err = err_new

    return c,l,err    

In [ ]:
#Kernel function 
def MultiRadialbasisfunc(Mean, Sigma, x):
    return np.sqrt(1/(2*pi*det(Sigma))) * exp((-1/2)*dot((x-Mean).T,dot(inv(Sigma),x-Mean)))

#activation fills the Phi (in the pdf)
def Multiactivation(X, Means, Sigma, numNeurons):
    # calculate activations of RBFs
    Phi = zeros((X.shape[0], numNeurons), float)
    for i in range(numNeurons):
        for j, x in enumerate(X):
            #Here we fill the element j,i with exp(-beta * (c_i-x_j)^2)
            Phi[j,i] = MultiRadialbasisfunc(Means[i], Sigma, x)
    return Phi


def Multitrainrbf(X, Y, Sigma, numNeurons, linear, Lambda):
    
    #Dimensions
    #X: n x Inputdimension
    #y: n x OutputDimension
    
    n = X.shape[0]
    d = Y.shape[1]
    
    #Centers are the neurons in our RBF network
    newCenters = kmeans(X,numNeurons)
    centers = newCenters[0]
    
    
    # calculate activations of RBFs
    if linear == 0:
        Phi = Multiactivation(X, centers, Sigma, numNeurons)
        Phi = np.append(Phi, X, axis =1)
        Phi = np.append(Phi, np.ones((n,1), "float"), axis = 1)
    else:
        Phi = X
        Phi = np.append(Phi, np.ones((n,1), "float"), axis = 1)
    
    
    # calculate output weights (pseudoinverse)
    #W = dot(pinv(Phi), Y) #written by Victor
    if linear == 0:
        W = dot(dot(inv(Lambda*np.identity(numNeurons+d+1)+dot(Phi.T, Phi)),Phi.T),Y)
    else:
        W = dot(dot(inv(Lambda*np.identity(d)+dot(Phi.T, Phi)),Phi.T),Y)
        
   
    
    #Calculate the covariance matrix Q with formula (6.18) 
    
    #Q = dot(np.transpose(Y),Y) - dot(np.transpose(W), dot(np.transpose(Phi), Y)) #written by Victor
    Q = np.zeros((d,d))
    for i in range(n):
        for k in range(d):
            for l in range(d):
                Q[k,l] += (1/float(n))*(Y[i,k] - dot(Phi[i,:],W[:,k]))*(Y[i,l] - dot(Phi[i,:],W[:,k]))
        
    #Q = (1/n)*Q
    
    return W, Q, centers